## Segmenting Clustering Neighborhoods in Toronto

---






### Import libraries:

In [0]:
import json
import folium
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from bs4 import BeautifulSoup
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.datasets.samples_generator import make_blobs

### Scrape the wiki page to extract the table and convert into Pandas dataframe:

In [51]:
webpage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(webpage.content,'lxml')
table = soup.find_all('table')[0]
postal_code_neighbourhood_df = pd.read_html(str(table), header=0)[0]
postal_code_neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned:

In [52]:
postal_code_neighbourhood_df = postal_code_neighbourhood_df[(postal_code_neighbourhood_df.Borough != 'Not assigned')]
postal_code_neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough:

So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park:


In [53]:
postal_code_neighbourhood_df.Neighbourhood.replace("Not assigned", postal_code_neighbourhood_df.Borough, inplace=True)
postal_code_neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


###Group the neighbourhood by the postal code: 

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table:

In [54]:
postal_code_group_neighbourhood_df = postal_code_neighbourhood_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
postal_code_group_neighbourhood_df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### Download the geospatial data into a data frame:

In [55]:
geo_data_url = "https://cocl.us/Geospatial_data"
!wget -q -O 'toronto_m.geospatial_data.csv' geo_data_url
postal_code_geo_df = pd.read_csv(geo_data_url)
postal_code_geo_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [56]:
postal_code_neighbourhood_geo_df = postal_code_neighbourhood_df.set_index('Postcode').join(postal_code_geo_df.set_index('Postal Code'))
postal_code_neighbourhood_geo_df = postal_code_neighbourhood_geo_df.reset_index()
postal_code_neighbourhood_geo_df.head(10)

,index,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [57]:
group_neighbourhood_geo_df = postal_code_group_neighbourhood_df.set_index('Postcode').join(postal_code_geo_df.set_index('Postal Code'))
group_neighbourhood_geo_df = group_neighbourhood_geo_df.reset_index()
group_neighbourhood_geo_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [58]:
print('The dataframe associated to Toronto has {} boroughs and {} neighbourhoods.'.format(
        len(postal_code_neighbourhood_df['Borough'].unique()),
        postal_code_neighbourhood_df.shape[0]
    )
)

The dataframe associated to Toronto has 11 boroughs and 211 neighbourhoods.


### Create the map of Toronto using latitude and longitude values





Use geopy library to get the latitude and longitude values of Toronto.

In [59]:
address = 'Toronto, ON'

geo_locator = Nominatim(user_agent="toronto_agent")
location = geo_locator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [1]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(postal_code_neighbourhood_geo_df['Latitude'], postal_code_neighbourhood_geo_df['Longitude'], postal_code_neighbourhood_geo_df['Borough'], postal_code_neighbourhood_geo_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'folium' is not defined

## Explore the neighbourhood data


Explore the first Borough in Toronto dataframe:

In [61]:
print('The first Borough in our dataframe is {} and has following neighbourhoods: {}'.format(group_neighbourhood_geo_df.loc[0, 'Borough'], group_neighbourhood_geo_df.loc[0, 'Neighbourhood']))

The first Borough in our dataframe is Scarborough and has following neighbourhoods: Rouge,Malvern


List of all neighbourhoods:

In [62]:
postal_code_neighbourhood_geo_df['Neighbourhood']

0                         Rouge
1                       Malvern
2                Highland Creek
3                    Rouge Hill
4                    Port Union
5                     Guildwood
6                   Morningside
7                     West Hill
8                        Woburn
9                     Cedarbrae
10          Scarborough Village
11         East Birchmount Park
12                      Ionview
13                 Kennedy Park
14                     Clairlea
15                  Golden Mile
16                     Oakridge
17                   Cliffcrest
18                    Cliffside
19     Scarborough Village West
20                  Birch Cliff
21               Cliffside West
22                  Dorset Park
23      Scarborough Town Centre
24              Wexford Heights
25                     Maryvale
26                      Wexford
27                    Agincourt
28               Clarks Corners
29                     Sullivan
                 ...           
181     

12th neighborhood's latitude and longitude values:

In [63]:
neighbourhood_latitude = postal_code_neighbourhood_geo_df.loc[11, 'Latitude'] 
neighbourhood_longitude = postal_code_neighbourhood_geo_df.loc[11, 'Longitude'] 

neighbourhood_name = postal_code_neighbourhood_geo_df.loc[11, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of East Birchmount Park are 43.7279292, -79.26202940000002.


First neighborhood's latitude and longitude values:

In [64]:
neighbourhood_latitude = postal_code_neighbourhood_geo_df.loc[0, 'Latitude'] 
neighbourhood_longitude = postal_code_neighbourhood_geo_df.loc[0, 'Longitude'] 

neighbourhood_name = postal_code_neighbourhood_geo_df.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))


Latitude and longitude values of Rouge are 43.806686299999996, -79.19435340000001.


Extract the information for Scarborough:

In [65]:
scarborough_df = postal_code_neighbourhood_geo_df[postal_code_neighbourhood_geo_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_df

,index,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get the latitude & longitude:

In [66]:
address = 'Scarborough, ON'

geo_locator = Nominatim(user_agent="scarborough_agent")
scarborough_location = geo_locator.geocode(address)
scarborough_latitude = scarborough_location.latitude
scarborough_longitude = scarborough_location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(scarborough_latitude, scarborough_longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


Create the map for Scarborough:

In [67]:
map_scarborough = folium.Map(location=[scarborough_latitude, scarborough_longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(scarborough_df['Latitude'], scarborough_df['Longitude'], scarborough_df['Borough'], scarborough_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

The url link to fetch the data from foursquare:

In [68]:
CLIENT_ID = 'O00P4X5EZLFEM5ETRB0NST55RTYRRURKDWEJ5YIEAXO4ORPM'
CLIENT_SECRET = 'I2DK0RRZSAKXVOCHCQMDSXB5ENBAJNTINSLWKHBDNQ0I5DML'
VERSION = '20180605'
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=O00P4X5EZLFEM5ETRB0NST55RTYRRURKDWEJ5YIEAXO4ORPM&client_secret=I2DK0RRZSAKXVOCHCQMDSXB5ENBAJNTINSLWKHBDNQ0I5DML&v=20180605&ll=43.653963,-79.387207&radius=500&limit=100'

In [69]:
url_contents = requests.get(url)
results = url_contents.json()
results

{'meta': {'code': 200, 'requestId': '5cc1fdc16a60712126e50f1e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 174,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

In [70]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
      
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(12)

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Tsujiri,Tea Room,43.655374,-79.385354
5,Poke Guys,Poke Place,43.654895,-79.385052
6,Manpuku まんぷく,Japanese Restaurant,43.653612,-79.390613
7,Rolltation,Japanese Restaurant,43.654918,-79.387424
8,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
9,Nathan Phillips Square,Plaza,43.652270,-79.383516


In [71]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

73 venues were returned by Foursquare.


Find the nearby venues:

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
toronto_venues = getNearbyVenues(names=group_neighbourhood_geo_df['Neighbourhood'],
                                   latitudes=group_neighbourhood_geo_df['Latitude'],
                                   longitudes=group_neighbourhood_geo_df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [74]:
scarborough_venues = getNearbyVenues(names=scarborough_df['Neighbourhood'],
                                   latitudes=scarborough_df['Latitude'],
                                   longitudes=scarborough_df['Longitude']
                                  )

Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge


In [75]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2250, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection


In [76]:
print(scarborough_venues.shape)
scarborough_venues.head(10)

(200, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
5,Rouge Hill,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
6,Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
7,Port Union,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
8,Guildwood,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
9,Guildwood,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Groupy the venues:

In [77]:
scarborough_venues_group = scarborough_venues.groupby('Neighbourhood')
scarborough_venues_group.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,Rouge Hill,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
5,Rouge Hill,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
6,Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
7,Port Union,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
8,Guildwood,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
9,Guildwood,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [78]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


One hot encoding for mechine learning:

In [79]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_onehot.head(12)

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant,Women's Store
0,Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Malvern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Highland Creek,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Highland Creek,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rouge Hill,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Rouge Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Port Union,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Port Union,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Guildwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Guildwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head(12)

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
print('Toronto: {}'.format(toronto_onehot.shape))
print('Scarborough: {}'.format(scarborough_onehot.shape))


Toronto: (2250, 278)
Scarborough: (200, 56)


In [82]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head(10)

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.010000,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.052632,0.0,0.0,0.00,0.0,0.0,0.0
6,Bayview Village,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
7,"Bedford Park,Lawrence Manor East",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
8,Berczy Park,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.017857,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
9,"Birch Cliff,Cliffside West",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0


In [83]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped.head(10)

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.000000,0.0
1,Agincourt North,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0
2,Birch Cliff,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.000000,0.0
3,Cedarbrae,0.000000,0.000000,0.142857,0.0,0.142857,0.142857,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.142857,0.000000,0.0
4,Clairlea,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.000000,0.000000,0.0
5,Clarks Corners,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.100000,0.000000,0.0
6,Cliffcrest,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0
7,Cliffside,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0
8,Cliffside West,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.000000,0.0
9,Dorset Park,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.166667,0.0


In [84]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04


----Agincourt----
               venue  freq
0             Lounge  0.25
1     Breakfast Spot  0.25
2     Clothing Store  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground  0.33
1           Asian Restaurant  0.33
2                       Park  0.33
3  Middle Eastern Restaurant  0.00
4        Monument / Landmark  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.25
1          Pizza Place  0.12
2           Beer Store  0.12
3  Fried Chicken Joint  0.12
4       Sandwich Place  0.12


----Alderwood,Long Branch----
            venue  f

In [85]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Clothing Store  0.25
1          Lounge  0.25
2  Breakfast Spot  0.25
3    Skating Rink  0.25
4        Pharmacy  0.00


----Agincourt North----
                venue  freq
0    Asian Restaurant  0.33
1                Park  0.33
2          Playground  0.33
3  Italian Restaurant  0.00
4   Korean Restaurant  0.00


----Birch Cliff----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0   Athletics & Sports  0.14
1               Bakery  0.14
2                 Bank  0.14
3      Thai Restaurant  0.14
4  Fried Chicken Joint  0.14


----Clairlea----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2          Park   0.1
3  Intersection   0.1
4  Soccer Field   0.1


----Clarks Corners----
                venue  freq
0         Pizza Place   0.2
1         

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighbourhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Sushi Restaurant,Restaurant,Burger Joint,Bar,Bakery
1,Agincourt,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Asian Restaurant,College Stadium,Colombian Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Dog Run,Dessert Shop,Dim Sum Restaurant
4,"Alderwood,Long Branch",Pizza Place,Pool,Coffee Shop,Sandwich Place,Gym,Pharmacy,Skating Rink,Pub,Dive Bar,Dessert Shop
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Sandwich Place,Deli / Bodega,Restaurant,Supermarket,Middle Eastern Restaurant,Bank,Fried Chicken Joint,Frozen Yogurt Shop,Diner
6,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Doner Restaurant,Discount Store,Dive Bar,Dog Run,Donut Shop
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Hardware Store,Butcher,Restaurant,Café,Pub,Pizza Place,Pharmacy
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Restaurant,Steakhouse,Pub,Cheese Shop,Seafood Restaurant
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [88]:
num_clusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 2, 4, 2, 2, 0, 0, 0, 0, 0], dtype=int32)

In [89]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = group_neighbourhood_geo_df
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Farmers Market
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,History Museum,Bar,Yoga Studio,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2.0,Intersection,Spa,Electronics Store,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Dog Run,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Cosmetics Shop,Concert Hall,Event Space
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2.0,Women's Store,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Discount Store,Department Store,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,2.0,Bus Line,Bakery,Intersection,Metro Station,Park,Fast Food Restaurant,Bus Station,Soccer Field,Discount Store,Dive Bar
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0.0,Movie Theater,Motel,American Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,College Stadium,Café,Skating Rink,General Entertainment,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


## Visualize the resulting clusters:

In [90]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##Examine each cluster and determine the discriminating venue categories that distinguish each cluster:

** Cluster 1**

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,0.0,Discount Store,Department Store,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
8,Scarborough,0.0,Movie Theater,Motel,American Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
9,Scarborough,0.0,College Stadium,Café,Skating Rink,General Entertainment,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
18,North York,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Electronics Store,Japanese Restaurant,Bakery,Food Court,Asian Restaurant,Tea Room,Women's Store
19,North York,0.0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Doner Restaurant,Discount Store,Dive Bar,Dog Run,Donut Shop
22,North York,0.0,Restaurant,Ramen Restaurant,Coffee Shop,Sandwich Place,Café,Shopping Mall,Discount Store,Japanese Restaurant,Plaza,Fast Food Restaurant
26,North York,0.0,Café,Baseball Field,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Gym / Fitness Center,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
27,North York,0.0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Clothing Store,Chinese Restaurant,Café,Restaurant,Dim Sum Restaurant,Discount Store
28,North York,0.0,Coffee Shop,Sandwich Place,Deli / Bodega,Restaurant,Supermarket,Middle Eastern Restaurant,Bank,Fried Chicken Joint,Frozen Yogurt Shop,Diner


** Cluster 2**

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Farmers Market


**Cluster 3**

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,History Museum,Bar,Yoga Studio,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,Scarborough,2.0,Intersection,Spa,Electronics Store,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Dog Run,Diner
4,Scarborough,2.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Cosmetics Shop,Concert Hall,Event Space
5,Scarborough,2.0,Women's Store,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
7,Scarborough,2.0,Bus Line,Bakery,Intersection,Metro Station,Park,Fast Food Restaurant,Bus Station,Soccer Field,Discount Store,Dive Bar
10,Scarborough,2.0,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Latin American Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
11,Scarborough,2.0,Middle Eastern Restaurant,Smoke Shop,Breakfast Spot,Shopping Mall,Bakery,Auto Garage,Sandwich Place,Yoga Studio,Dog Run,Discount Store
12,Scarborough,2.0,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
13,Scarborough,2.0,Pizza Place,Fried Chicken Joint,Noodle House,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Pharmacy,Donut Shop
15,Scarborough,2.0,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Electronics Store,Burger Joint,Breakfast Spot,Sandwich Place,Pharmacy,Pizza Place,Coffee Shop


**Cluster 4** 

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,3.0,Park,Bank,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
94,Etobicoke,3.0,Bank,Yoga Studio,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant


**Cluster 5** 

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4.0,Park,Playground,Asian Restaurant,College Stadium,Colombian Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
25,North York,4.0,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
30,North York,4.0,Park,Bus Stop,Airport,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
40,East York,4.0,Convenience Store,Park,Coffee Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
44,Central Toronto,4.0,Park,Bus Line,Swim School,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dessert Shop
50,Downtown Toronto,4.0,Park,Playground,Trail,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
64,Central Toronto,4.0,Park,Jewelry Store,Sushi Restaurant,Trail,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
72,North York,4.0,Park,Pizza Place,Japanese Restaurant,Pub,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
74,York,4.0,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
79,North York,4.0,Park,Massage Studio,Construction & Landscaping,Bakery,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
